In [1]:
import csv
import numpy as np
import pandas as pd
import GPy
import seaborn as sns
sns.set_style('white')
sns.set_context('talk')
import pickle

In [4]:
with open('targets.csv') as f:
    reader = csv.reader(f)
    targets_all = next(reader)
    
print('Total '+str(len(targets_all))+' targets:')
print(targets_all)

Total 26 targets:
['AVG_DEMAND_KW_CALCULATED_12', 'NSN_LF59_BLDG-CDW-LOOP_CDW-FLOW', 'NSN_LF59_BLDG-CDW-LOOP_CDW-T-R', 'NSN_LF59_BLDG-CDW-LOOP_CDW-T-S', 'NSN_LF59_COOLING-TOWER_FAN-HI', 'NSN_LF59_COOLING-TOWER_FAN-LO', 'NSN_LF59_COOLING-TOWER_FAN-STAT', 'NSN_LF59_CWP-1-VFD_VFD-SIG', 'NSN_LF59_CWP-1_PMP', 'NSN_LF59_CWP-1_PMP-STAT', 'NSN_LF59_BOILER-1_HW-T-E', 'NSN_LF59_BOILER-1_HW-T-L', 'NSN_LF59_PWSHP-2_CLG-COIL-T', 'NSN_LF59_PWSHP-2_OA-FLOW', 'NSN_LF59_PWSHP-2_SA-T', 'NSN_LF59_PWSHP-2_SA-T-STPT', 'NSN_LF59_WSHP-01_DA-T', 'NSN_LF59_WSHP-01_LW-T', 'NSN_LF59_WSHP-01_ZN-STPT-CL-EFF', 'NSN_LF59_WSHP-01_ZN-STPT-HT-EFF', 'NSN_LF59_WSHP-01_ZN-T', 'NSN_LF59_WSHP-12_DA-T', 'NSN_LF59_WSHP-12_LW-T', 'NSN_LF59_WSHP-12_ZN-STPT-CL-EFF', 'NSN_LF59_WSHP-12_ZN-STPT-HT-EFF', 'NSN_LF59_WSHP-12_ZN-T']


In [10]:
path_to_data = './data/'
year = '2020'
# targets = targets_all
# targets = ['AVG_DEMAND_KW_CALCULATED_12','NSN_LF59_BLDG-CDW-LOOP_CDW-FLOW','NSN_LF59_BLDG-CDW-LOOP_CDW-T-R','NSN_LF59_BLDG-CDW-LOOP_CDW-T-S','NSN_LF59_COOLING-TOWER_FAN-HI','NSN_LF59_COOLING-TOWER_FAN-LO','NSN_LF59_COOLING-TOWER_FAN-STAT','NSN_LF59_CWP-1-VFD_VFD-SIG','NSN_LF59_CWP-1_PMP','NSN_LF59_CWP-1_PMP-STAT','NSN_LF59_BOILER-1_HW-T-E','NSN_LF59_BOILER-1_HW-T-L','NSN_LF59_PWSHP-2_CLG-COIL-T','NSN_LF59_PWSHP-2_OA-FLOW','NSN_LF59_PWSHP-2_SA-T','NSN_LF59_PWSHP-2_SA-T-STPT','NSN_LF59_WSHP-01_DA-T','NSN_LF59_WSHP-01_LW-T','NSN_LF59_WSHP-01_ZN-STPT-CL-EFF','NSN_LF59_WSHP-01_ZN-STPT-HT-EFF','NSN_LF59_WSHP-01_ZN-T','NSN_LF59_WSHP-12_DA-T','NSN_LF59_WSHP-12_LW-T','NSN_LF59_WSHP-12_ZN-STPT-CL-EFF','NSN_LF59_WSHP-12_ZN-STPT-HT-EFF','NSN_LF59_WSHP-12_ZN-T','NSN_LF59_BuildingKWhdSum','NSN_LF59_BuildingKWHdSumAnnualized']
targets = [targets_all[2]]
for target in targets:
    print('Target: '+target)

datafilename = 'train_data_'+year+'.csv'
data = pd.read_csv(path_to_data+datafilename)

X = np.vstack((data.index.to_numpy(), data['HOD'].to_numpy(), data['Environment:Site Outdoor Air Drybulb Temperature [C](TimeStep)'].to_numpy())).T
X[:,0] = X[:,0] / 35040.0
X[:,1] = X[:,1] / 24.0
X[:,2] = (X[:,2]+20.0) / 60.0

x_train = X

Target: NSN_LF59_BLDG-CDW-LOOP_CDW-T-R


In [11]:
for target in targets:
    print('Working on ' + target + '...')
    y_train = data[target].to_numpy()
    y_train = y_train[:, None]
    print(x_train.shape)
    print(y_train.shape)
    k = GPy.kern.RBF(x_train.shape[1], ARD=True) # The ARD = True is what makes GPy understand that there is
                                # one lengthscale per dimension
    m = GPy.models.GPRegression(x_train, y_train, k)
    print(m)

    m.optimize_restarts( robust=True )
    print(m)
    print(m.rbf.lengthscale)

    with open('./model/'+target+'.pkl', 'wb') as file:
        pickle.dump(m, file)

    print('Model for ' + target + ' Complete!')

Working on NSN_LF59_BLDG-CDW-LOOP_CDW-T-R...
(16580, 3)
(16580, 1)

Name : GP regression
Objective : 1407068.4773158664
Number of Parameters : 5
Number of Optimization Parameters : 5
Updates : True
Parameters:
  GP_regression.           |  value  |  constraints  |  priors
  rbf.variance             |    1.0  |      +ve      |        
  rbf.lengthscale          |   (3,)  |      +ve      |        
  Gaussian_noise.variance  |    1.0  |      +ve      |        
Warning - optimization restart 1/10 failed
Warning - optimization restart 2/10 failed
Warning - optimization restart 3/10 failed
Warning - optimization restart 4/10 failed
Warning - optimization restart 5/10 failed
Warning - optimization restart 6/10 failed
Warning - optimization restart 7/10 failed
Warning - optimization restart 8/10 failed
Warning - optimization restart 9/10 failed
